In [1]:
import numpy as np
from collections import defaultdict
import random
from numpy import random
import networkx as nx
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import gurobipy as gp
from gurobipy import GRB
from gurobipy import*
from gurobipy import quicksum

import pickle

## Quantum Part CQM
from dwave.system import DWaveSampler, LeapHybridSampler, LeapHybridCQMSampler
from dwave.system.composites import EmbeddingComposite
import dwave.inspector
import dimod

from dimod.binary.binary_quadratic_model import BinaryQuadraticModel, Binary, Spin, as_bqm
from dimod.discrete.discrete_quadratic_model import DiscreteQuadraticModel

## old setup
$$
\begin{equation*}
\begin{array}{lll}
&\displaystyle \min_{\substack{
u^{\text{dis}}_{t},\ u^{\text{chr}}_{t},p^{\text{dis}}_{t},\ p^{\text{chr}}_{t},x^{\text{chiller}}_{j,t},\ x^{\text{tower}}_{j,t} \\
T_{i,t}^{\text{Zone}},\ T_{i,t+1}^{\text{Zone}} ,T_{i,t}^{\text{sup}},\ v^{\text{vent}}_{t} \\}} 
\sum_{t=0}^{T}p^{e,g}_{t} e^{\text{dc,in}}_{t} & \\
s.t.\quad &e^{\text{dc,in}}_{t} = E^{\text{HVAC}}_t + E^{\text{Se}}_{t} +\Delta E^{\text{B}}_{t} - E^{\text{Solar}}_{t} & \forall  t \in \mathbf{T}\\
&e^{\text{dc,in}}_{t} \geq 0 & \forall t \in \mathbf{T}\\
& \Delta E^{\text{B}}_{t} = p^{\text{chr}}_{t}\eta^{\text{chr}} - p^{\text{dis}}_{t} \cdot (\eta^{\text{dis}})^{-1} & \forall  t \in \mathbf{T}\\ 
& E^{\text{B,state}}_{t+1} = E^{\text{B,state}}_{t} + \Delta E^{\text{B}}_{t} & \forall  t \in \mathbf{T} \\
& \underline{\xi^{\text{B}}} \leq E^{\text{B,state}}_{t} \leq \overline{\xi^{\text{B}}} & \forall  t \in \mathbf{T} \\
& p^{\text{chr}}_{t} \leq \overline{p^{\text{chr}}_{t}} \cdot u^{\text{chr}}_{t} & \forall  t \in \mathbf{T}\\
& p^{\text{dis}}_{t} \leq \overline{p^{\text{dis}}_{t}} \cdot u^{\text{dis}}_{t} & \forall  t \in \mathbf{T}\\
& u^{\text{chr}}_{t} + u^{\text{dis}}_{t} \leq 1 & \forall  t \in \mathbf{T}\\
&T^{\text{Zone}}_{i,t+1} = T^{\text{Zone}}_{i,t} 
+ \displaystyle\sum_{i^{\prime} \in  \mathcal{N}\left(i\right)} 
\left(\frac{T_{i^{\prime},t}^{\text{Zone}} - T_{i,t}^{\text{Zone}}}
{C^{\text{heat}}_{i}R^{\text{Zone}}_{i^{\prime}i}}\right)
+ \frac{\dot{m}_{i} c_{a}\left(T_{i,t}^{\text{sup}} - T_{i,t}^{\text{Zone}}\right) 
+ \theta_{i,t}}{C^{\text{heat}}_{i}} & \forall i \in \mathbf{I}^{\text{Zone}}, t \in \mathbf{T}\\
&T_{i,t}^{-} \leq T^{\text{Zone}}_{i,t} \leq T_{i,t}^{+}, & \forall i \in \mathbf{I}^{\text{Zone}}, t \in \mathbf{T} \\
&v^{\text{vent}}_t + v^{\text{out}}_t \geq \underline{v}^{\text{vent}}_t & \forall  t \in \mathbf{T}\\
&L^{\text{heat}}_{t} = \left(T^{\text{out}}_{t}-T^{\text{sup}}_{t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \left(\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }{\chi_{i}T^{\text{Zone}}_{i,t}}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p & \forall t \in \mathbf{T}\\
&\displaystyle \sum_{j \in \mathbf{J}^{\text{chiller}}} x^{\text{chiller}}_{j,t}m^{\text{chiller}}_{j,t} \geq \frac{\left(T^{\text{out}}_{t}-T^{\text{sup}}_{t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \left(\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }{\chi_{i}T^{\text{Zone}}_{i,t}}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}  & \forall t \in \mathbf{T}\\
&\displaystyle \sum_{j \in \mathbf{J}^{\text{tower}}}{x^{\text{tower}}_{j,t}m^{\text{tower}}_{j,t}} \geq \frac{\left(T^{\text{out}}_{t}-T^{\text{sup}}_{t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \left(\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }{\chi_{i}T^{\text{Zone}}_{i,t}}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p }
{\left(T^{\text{conwr}}_{t}-T^{\text{conws}}_{t}\right)\cdot  c^{\text{water}}_p }  & \forall t \in \mathbf{T}\\
&e^{\text{vent}}_{t}=\beta^{\text{vent}}_{0}+\beta^{\text{vent}}_{1}\left(v^{\text{vent}}_t-\underline{v}^{\text{vent}}\right) 
& \forall t \in \mathbf{T}\\
&e^{\text{chiller}}_{t}=\displaystyle\sum_{j \in \mathbf{J}^{\text{chiller}}}x^{\text{chiller}}_{j,t}\left(\beta^{\text{chiller}}_{0,j}
 + \beta^{\text{chiller}}_{1,j}m^{\text{chiller}}_{j,t}\right) & \forall t \in \mathbf{T}\\
&e^{\text {pump }}_{t}=\beta^{\text{pump}}_{0}+\beta^{\text{pump}}_{1} \frac{\left(T^{\text{out}}_{t}-T^{\text{sup}}_{t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \left(\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }{\chi_{i}T^{\text{Zone}}_{i,t}}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}  & \forall t \in \mathbf{T}\\
&e^{\text{tower}}_{t} = \displaystyle\sum_{j \in \mathbf{J}^{\text{tower}}}x^{\text{tower}}_{j,t} \left(\beta^{\text{tower}}_{0,j} + \beta^{\text{tower}}_{1,j}m^{\text{tower}}_{j,t}\right) & \forall t \in \mathbf{T}\\
&E^{\text{HVAC}}_{t} =  e^{\text{sup}}_{t} + e^{\text{vent}}_{t}+ e^{\text{chiller}}_{t} + e^{\text{pump}}_{t} + e^{\text{tower}}_{t} & \forall  t \in \mathbf{T}\\
\end{array}
\end{equation*}
$$

## New Setup

$$
\begin{equation*}
\begin{array}{lll}
&\displaystyle \min_{\substack{
u^{\text{dis}}_{t},\ u^{\text{chr}}_{t},p^{\text{dis}}_{t},\ p^{\text{chr}}_{t},x^{\text{chiller}}_{j,t},\ x^{\text{tower}}_{j,t} \\
T_{i,t}^{\text{Zone}},\ T_{i,t+1}^{\text{Zone}} ,T_{i,t}^{\text{sup}},\ v^{\text{vent}}_{t} \\}} 
\sum_{t=0}^{T}p^{e,g}_{t} e^{\text{dc,in}}_{t} & \\
s.t.\quad &e^{\text{dc,in}}_{t} = E^{\text{HVAC}}_t + E^{\text{Se}}_{t} +\Delta E^{\text{B}}_{t} - E^{\text{Solar}}_{t} & \forall  t \in \mathbf{T}\\
&e^{\text{dc,in}}_{t} \geq 0 & \forall t \in \mathbf{T}\\
& \Delta E^{\text{B}}_{t} = p^{\text{chr}}_{t}\eta^{\text{chr}} - p^{\text{dis}}_{t} \cdot (\eta^{\text{dis}})^{-1} & \forall  t \in \mathbf{T}\\ 
& E^{\text{B,state}}_{t+1} = E^{\text{B,state}}_{t} + \Delta E^{\text{B}}_{t} & \forall  t \in \mathbf{T} \\
& \underline{\xi^{\text{B}}} \leq E^{\text{B,state}}_{t} \leq \overline{\xi^{\text{B}}} & \forall  t \in \mathbf{T} \\
& p^{\text{chr}}_{t} \leq \overline{p^{\text{chr}}_{t}} \cdot u^{\text{chr}}_{t} & \forall  t \in \mathbf{T}\\
& p^{\text{dis}}_{t} \leq \overline{p^{\text{dis}}_{t}} \cdot u^{\text{dis}}_{t} & \forall  t \in \mathbf{T}\\
& u^{\text{chr}}_{t} + u^{\text{dis}}_{t} \leq 1 & \forall  t \in \mathbf{T}\\
&T^{\text{Zone}}_{i,t+1} = T^{\text{Zone}}_{i,t} 
+ \displaystyle\sum_{i^{\prime} \in  \mathcal{N}\left(i\right)} 
\left(\frac{T_{i^{\prime},t}^{\text{Zone}} - T_{i,t}^{\text{Zone}}}
{C^{\text{heat}}_{i}R^{\text{Zone}}_{i^{\prime}i}}\right)
+ \frac{\dot{m}_{i} c_{a}\left(T_{i,t}^{\text{sup}} - T_{i,t}^{\text{Zone}}\right) 
+ \theta_{i,t}}{C^{\text{heat}}_{i}} & \forall i \in \mathbf{I}^{\text{Zone}}, t \in \mathbf{T}\\
&T_{i,t}^{-} \leq T^{\text{Zone}}_{i,t} \leq T_{i,t}^{+}, & \forall i \in \mathbf{I}^{\text{Zone}}, t \in \mathbf{T} \\
&v^{\text{vent}}_t + v^{\text{out}}_t \geq \underline{v}^{\text{vent}}_t & \forall  t \in \mathbf{T}\\
&\displaystyle \sum_{j \in \mathbf{J}^{\text{chiller}}} x^{\text{chiller}}_{j,t}m^{\text{chiller}}_{j,t} \geq \frac{\left(T^{\text{out}}_{t}-\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}}}\chi_{i}T^{\text{sup}}_{i,t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i}\left(T^{\text{Zone}}_{i,t}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}  & \forall t \in \mathbf{T}\\
&\displaystyle \sum_{j \in \mathbf{J}^{\text{tower}}}{x^{\text{tower}}_{j,t}m^{\text{tower}}_{j,t}} \geq \frac{\left(T^{\text{out}}_{t}-\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}}}\chi_{i}T^{\text{sup}}_{i,t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i}\left(T^{\text{Zone}}_{i,t}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p }
{\left(T^{\text{conwr}}_{t}-T^{\text{conws}}_{t}\right)\cdot  c^{\text{water}}_p }  & \forall t \in \mathbf{T}\\
&e^{\text{vent}}_{t}=\beta^{\text{vent}}_{0}+\beta^{\text{vent}}_{1}\left(v^{\text{vent}}_t-\underline{v}^{\text{vent}}\right) 
& \forall t \in \mathbf{T}\\
&e^{\text{chiller}}_{t}=\displaystyle\sum_{j \in \mathbf{J}^{\text{chiller}}}x^{\text{chiller}}_{j,t}\left(\beta^{\text{chiller}}_{0,j}
 + \beta^{\text{chiller}}_{1,j}m^{\text{chiller}}_{j,t}\right) & \forall t \in \mathbf{T}\\
&e^{\text {pump }}_{t}=\beta^{\text{pump}}_{0}+\beta^{\text{pump}}_{1} \frac{\left(T^{\text{out}}_{t}-T^{\text{sup}}_{t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i}\left(T^{\text{Zone}}_{i,t}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}  & \forall t \in \mathbf{T}\\
&e^{\text{tower}}_{t} = \displaystyle\sum_{j \in \mathbf{J}^{\text{tower}}}x^{\text{tower}}_{j,t} \left(\beta^{\text{tower}}_{0,j} + \beta^{\text{tower}}_{1,j}m^{\text{tower}}_{j,t}\right) & \forall t \in \mathbf{T}\\
&E^{\text{HVAC}}_{t} =  e^{\text{sup}}_{t} + e^{\text{vent}}_{t}+ e^{\text{chiller}}_{t} + e^{\text{pump}}_{t} + e^{\text{tower}}_{t} & \forall  t \in \mathbf{T}\\
\end{array}
\end{equation*}
$$


In [2]:
class DC_model:
    
    def __init__(self, random_choice = 0, T = 3, J_chiller =4, J_tower = 5,  **kwargs):
        # Total time interval
        self.T = T

        # time interval length
        self.delta_t = 5 # minutes

        # Total Data center zones
        self.I_zone_wide = 3
        self.I_zone_length = 3
        self.I_zone =  self.I_zone_wide * self.I_zone_length

        # time interval length
        self.delta_t = 5 # minutes

        # Real time grid electricity price
        #https://www.energybot.com/electricity-rates/texas/
        if random_choice:
            self.mu = 0.989
            self.sigma = 0.001
            self.p_eg = np.random.normal(self.mu, self.sigma, self.T + 1)
        else:
            self.p_eg = np.arange(1, self.T + 1, 1, dtype=int) * 0.989
        
        # Batter electricity charging upper bound
        self.char_b_upperbound = 60 * self.delta_t / 60

        # Batter electricity discharging upper bound
        self.dis_b_upperbound = 90 * self.delta_t / 60

        # Battery electricity capacity
        self.Battery_capacity = 150 #  unit kWh

        # Battery initial energy
        self.Battery_capacity_init = 0.5 * self.Battery_capacity #  unit kWh

        # Battery electricity capacity uppper and lower alert
        self.Battery_capacity_upbound = 0.85 * self.Battery_capacity
        self.Battery_capacity_lowbound = 0.15 * self.Battery_capacity 

        #battery charging efficiency
        self.eta_char_b = 0.9

        #battery discharging efficiency
        self.eta_dischar_b = 1 / 0.85

        #zone tempreture initialization celcius or fahrenheit
        self.T_initial_upperbound = 25
        self.T_initial_lowerbound = 33 

        if random_choice == 0 :
            self.T_zone_initial =np.array([[31, 30, 29],[32, 26, 26],[30, 24, 27]])
            # Outside air temperature
            self.T_out = np.array([30,34,32])  # unit K
        else:
            self.T_zone_initial = np.random.randint(25, 33, (self.I_zone_wide, self.I_zone_length))
            # Outside air temperature
            self.T_out = np.random.randint(27, 36, size = self.T) # unit Celcious
            
            
        #Zone space 
        if random_choice == 0 :
            self.Zone_space = np.array([[1020,  950,  950],[1010, 1060, 1040],[ 900, 1050,  930]])
        else:
            self.Zone_space = np.random.randint(900, 1100, (self.I_zone_wide, self.I_zone_length))
            
        self.DC_height = 6 # meters
        
        # sever electricity consumpsion
        self.e_server = 60 * np.ones(self.T)  #kWh

        # Solar electricity production
        if random_choice == 0 :
            self.e_solar = np.array([65, 85, 40]) * self.delta_t / 60   #kWh
        else:
            self.e_solar = np.random.randint(40, 150, size=self.T) * self.delta_t / 60

        self.DC_height = 6 # meters
        
        
        # sever electricity consumpsion
        # with a total U value of 0.22, lighting internal gains modelled considering fluorescent lighting with 7 W/m2 intensity, 
        # and IT equipment and servers modelled with 2500 W/m2 intensity.

        self.e_server = self.delta_t / 60 * 2.5 * np.ones(self.T) * np.sum(self.Zone_space)  #kWh
        
        # zone air
        ## air thermal resistance value (0.4-0.6)
        ## https://www-sciencedirect-com.ezproxy.lib.uh.edu/topics/engineering/thermal-resistance
        ## https://fscdn.rohm.com/en/products/databook/applinote/common/basics_of_thermal_resistance_and_heat_dissipation_an-e.pdf
        ## http://www.sfu.ca/phys/346/121/lecture_notes/lecture33_heat_loss.pdf

        self.R_air_ij = 0.062  #unit m.K/W

        # air heat capacity in room i
        #C_air =  718 * 1.225 * 100 * np.ones(self.I_zone) # unit J/K  rho = 1.225 kg/m3

        # AC wind speed
        self.v_sup = np.ones(self.T) * 1 * self.delta_t * 60 # unit m/s

        # air mass flow into the room i
        self.m_air_in =   4 * np.kron(self.v_sup.reshape(1,-1)  , np.ones((self.I_zone,1)) ) * 1.2  # unit m2 * m/s * Kg/m^3 * s

        # specific heat capacity of air in room i 
        self.c_s_air = 0.718 #unit [KJ/( kg · K )]

        # internal heat generation of server radiated to room i
        self.theta_heat =  self.delta_t / 60 * 2.5 * np.kron(np.ones((1,self.T)) , self.Zone_space.reshape(-1,1)) #unit W/K

        # outside wind speed
        self.v_out = np.ones(self.T) * 0.5 * self.delta_t * 60 # unit m/s

        # AC wind power
        self.beta_sup = 5 # unit kWh/(m/s)
        self.e_sup = self.beta_sup * self.v_sup # unit kWh

        # ventilation air flow rate lower bound
        self.v_vent_lowerbound = np.ones(self.T) * 0.8 * self.delta_t * 60 # unit m/s

        # heat capacity of air.
        self.C_air_room =  0.718 * 1.2 * self.Zone_space * self.DC_height  # unit [KJ/( kg · K )] *  kg/m3 * m3  = KJ/K
        self.C_air_room_flat = self.C_air_room.flatten()
         
        # heat capacity of water.
        self.C_water =  4.182  # unit [KJ/( kg · K )]

        # Room air temperature upper bound
        self.T_datacenter_upperbound = np.ones((self.I_zone,self.T)) * 24

        # Room air temperature lower bound
        self.T_datacenter_lowerbound = np.ones((self.I_zone,self.T)) * 15

        # Room AC temperature upper bound
        self.T_AC_upperbound = np.ones((self.I_zone,self.T)) * 26

        # Room AC temperature lower bound
        self.T_AC_lowerbound = np.ones((self.I_zone,self.T)) * 10

        # chiller water temperature supply
        self.T_chiller_water_supply = np.ones(self.T) * 20

        # chiller water temperature return
        self.T_chiller_water_return = np.ones(self.T) * 30

        # condense water temperature supply
        self.T_tower_water_supply = self.T_out

        # condense water temperature return
        self.T_tower_water_return = np.ones(self.T) * 41

        # ventilation air flow rate start up power
        self.beta_vent_0 = 100 # unit kJ
        self.beta_vent_1 = 10 # unit kJ s/m
        self.const_vent = self.beta_vent_0  - self.beta_vent_1 * self.v_vent_lowerbound

        # pump water flow rate power
        self.beta_pump_0 = 100 # unit kJ
        self.beta_pump_1 = 10 # unit kJ/(kg)

        # chiller amount
        self.J_chiller = J_chiller

        # tower amount 
        self.J_tower = J_tower

        # chiller rated water flow rate
        # tower rated water flow rate
        if random_choice == 0 :
            self.M_chiller_rated = np.array([60, 100, 140, 180]) # unit kg/t
            self.M_tower_rated = np.array([50, 75, 100, 125, 150]) # unit kg/t   
        else:
            self.M_chiller_rated = np.random.randint(60, 160, size=self.J_chiller) * self.J_chiller / 4 # unit kg/t
            self.M_tower_rated = np.random.randint(50, 150, size=self.J_tower) * self.J_tower / 5 # unit kg/t   

        # chiller rated power
        self.beta_chiller_0 = 4 * np.ones(self.J_chiller) * self.J_chiller / 4 # unit kWh
        self.beta_chiller_1 = 7 * np.ones(self.J_chiller) # unit kWh/(kg)

        # tower rated power
        self.beta_tower_0 = 6 * np.ones(self.J_tower) * self.J_tower / 5 # unit kWh
        self.beta_tower_1 = 5 * np.ones(self.J_tower) # unit kWh/(kg)

        # weight of each zone in data center
        self.Chi = np.ones(self.I_zone) * self.Zone_space.flatten() / np.sum(self.Zone_space)

        #adjacent zone map (default)
        #4|5|6|
        #1|2|3|
        self.adjacent_zone_map = np.zeros((self.I_zone,self.I_zone))
        for zone in range(self.I_zone):
            zone_x  = zone // self.I_zone_length
            zone_y = zone % self.I_zone_length
            
            adjacent_zone_x = np.array([zone_x-1, zone_x+1])
            adjacent_zone_y = np.array([zone_y-1, zone_y+1])
            
            adjacent_zone_x_update = adjacent_zone_x[ (adjacent_zone_x >= 0) & (adjacent_zone_x < self.I_zone_wide) ]
            adjacent_zone_y_update = adjacent_zone_y[ (adjacent_zone_y >= 0) & (adjacent_zone_y < self.I_zone_length) ]
            
        # print(adjacent_zone_x, adjacent_zone_y)
            
            self.adjacent_zone_map[zone, adjacent_zone_x_update * self.I_zone_length + zone_y] = 1
            self.adjacent_zone_map[zone, zone_x * self.I_zone_length + + adjacent_zone_y_update] = 1
            
        #print(adjacent_zone_map)
        self.K_chiller = self.c_s_air / ((self.T_chiller_water_return - self.T_chiller_water_supply) * self.C_water)

        self.K_tower = self.c_s_air / ((self.T_tower_water_return - self.T_tower_water_supply) * self.C_water)

        #const pump
        self.const_pump =  self.beta_pump_0  + self.beta_pump_1 * self.K_chiller * self.T_out * self.v_out 

#print(self.adjacent_zone_map)

In [3]:
#DCmodel = DC_model(random_choice = 1, T = 24, J_chiller =2, J_tower = 2)

with open(f'Testbench/test_T_4_JC_2_JT_2', 'rb') as file2:
    DCmodel = pickle.load(file2)

In [4]:
# Total time interval
T = DCmodel.T

# Total Data center zones
I_zone_wide = DCmodel.I_zone_wide
I_zone_length = DCmodel.I_zone_wide
I_zone =  DCmodel.I_zone

# time interval length
delta_t = DCmodel.delta_t # minutes

# Real time grid electricity price
p_eg = DCmodel.p_eg

# Batter electricity charging upper bound
char_b_upperbound = DCmodel.char_b_upperbound

# Batter electricity discharging upper bound
dis_b_upperbound = DCmodel.dis_b_upperbound 

# Battery electricity capacity
Battery_capacity = DCmodel.Battery_capacity  #  unit kWh

# Battery initial energy
Battery_capacity_init = DCmodel.Battery_capacity_init  #  unit kWh

# Battery electricity capacity uppper and lower alert
Battery_capacity_upbound = DCmodel.Battery_capacity_upbound 
Battery_capacity_lowbound = DCmodel.Battery_capacity_lowbound

#battery charging efficiency
eta_char_b = DCmodel.eta_char_b

#battery discharging efficiency
eta_dischar_b = DCmodel.eta_dischar_b

#zone tempreture initialization celcius or fahrenheit
T_initial_upperbound = DCmodel.T_initial_upperbound
T_initial_lowerbound = DCmodel.T_initial_upperbound


T_zone_initial = DCmodel.T_zone_initial
# Outside air temperature
T_out = DCmodel.T_out  # unit K
    
#Zone space 
Zone_space = DCmodel.Zone_space

# Solar electricity production
e_solar = DCmodel.e_solar  #kWh
    
DC_height = DCmodel.DC_height # meters

# sever electricity consumpsion
#with a total U value of 0.22, lighting internal gains modelled considering fluorescent lighting with 7 W/m2 intensity, and IT equipment and servers modelled with 2500 W/m2 intensity.
e_server = DCmodel.e_server #kWh

# zone air
## air thermal resistance value (0.4-0.6)
## https://www-sciencedirect-com.ezproxy.lib.uh.edu/topics/engineering/thermal-resistance
## https://fscdn.rohm.com/en/products/databook/applinote/common/basics_of_thermal_resistance_and_heat_dissipation_an-e.pdf
## http://www.sfu.ca/phys/346/121/lecture_notes/lecture33_heat_loss.pdf

R_air_ij = DCmodel.R_air_ij #unit m.K/W

# air heat capacity in room i
#C_air =  718 * 1.225 * 100 * np.ones(I_zone) # unit J/K  rho = 1.225 kg/m3

# AC wind speed
v_sup = DCmodel.v_sup # unit m/s

# air mass flow into the room i
m_air_in = DCmodel.m_air_in  # unit m2 * m/s * Kg/m^3 * s

# specific heat capacity of air in room i 
c_s_air = DCmodel.c_s_air #unit [KJ/( kg · K )]

# internal heat generation of server radiated to room i
theta_heat = DCmodel.theta_heat #unit W/K

# outside wind speed
v_out = DCmodel.v_out # unit m/s

# AC wind power
beta_sup = DCmodel.beta_sup # unit kWh/(m/s)
e_sup = DCmodel.e_sup # unit kWh

# ventilation air flow rate lower bound
v_vent_lowerbound = DCmodel.v_vent_lowerbound  # unit m/s

# heat capacity of air.
C_air_room = DCmodel.C_air_room  # unit [KJ/( kg · K )] *  kg/m3 * m3  = KJ/K
C_air_room_flat = DCmodel.C_air_room_flat

# heat capacity of water.
C_water = DCmodel.C_water  # unit [KJ/( kg · K )]

# Room air temperature upper bound
T_datacenter_upperbound = DCmodel.T_datacenter_upperbound 

# Room air temperature lower bound
T_datacenter_lowerbound = DCmodel.T_datacenter_lowerbound 

# Room AC temperature upper bound
T_AC_upperbound = DCmodel.T_AC_upperbound

# Room AC temperature lower bound
T_AC_lowerbound = DCmodel.T_AC_lowerbound

# chiller water temperature supply
T_chiller_water_supply = DCmodel.T_chiller_water_supply

# chiller water temperature return
T_chiller_water_return = DCmodel.T_chiller_water_return

# condense water temperature supply
T_tower_water_supply = DCmodel.T_tower_water_supply

# condense water temperature return
T_tower_water_return = DCmodel.T_tower_water_return

# ventilation air flow rate start up power
beta_vent_0 = DCmodel.beta_vent_0 # unit kJ
beta_vent_1 = DCmodel.beta_vent_1 # unit kJ s/m
const_vent = DCmodel.const_vent

# pump water flow rate power
beta_pump_0 = DCmodel.beta_pump_0  # unit kJ
beta_pump_1 = DCmodel.beta_pump_1 # unit kJ/(kg)


# chiller amount
J_chiller = DCmodel.J_chiller

# tower amount 
J_tower = DCmodel.J_tower

# chiller rated water flow rate
M_chiller_rated = DCmodel.M_chiller_rated # unit kg/t
M_tower_rated = DCmodel.M_tower_rated # unit kg/t   
 

# chiller rated power
beta_chiller_0 = DCmodel.beta_chiller_0 # unit kWh
beta_chiller_1 = DCmodel.beta_chiller_1  # unit kWh/(kg)

# tower rated power
beta_tower_0 = DCmodel.beta_tower_0 # unit kWh
beta_tower_1 = DCmodel.beta_tower_1 # unit kWh/(kg)

# weight of each zone in data center
Chi = DCmodel.Chi

#adjacent zone map (default)
#4|5|6|
#1|2|3|
adjacent_zone_map = DCmodel.adjacent_zone_map

#print(adjacent_zone_map)
K_chiller = DCmodel.K_chiller

K_tower = DCmodel.K_tower 

#const pump
const_pump = DCmodel.const_pump